In [1]:
# Import the libraries we will need
import pandas as pd
import numpy as np

import findspark
findspark.init()

from pyspark.sql import *
import pyspark.sql.functions as F
from pyspark.sql.types import *

# create the Spark Session
spark = SparkSession.builder.appName("Q2").getOrCreate()

In [20]:
small = spark.sparkContext.textFile("hw3-bundle/hw3-bundle/pagerank_hits/data/graph-small.txt")
# small = spark.read.csv("hw3-bundle/hw3-bundle/pagerank_hits/data/graph-small.txt", sep="\t", inferSchema=True).toDF("source", "destination")

In [21]:
small.take(10)

['100\t1',
 '13\t1',
 '28\t1',
 '89\t1',
 '82\t1',
 '30\t1',
 '79\t1',
 '65\t1',
 '88\t1',
 '25\t1']

In [24]:
small = small.map(lambda x: x.split())

In [26]:
small.take(10)

[['100', '1'],
 ['13', '1'],
 ['28', '1'],
 ['89', '1'],
 ['82', '1'],
 ['30', '1'],
 ['79', '1'],
 ['65', '1'],
 ['88', '1'],
 ['25', '1']]

In [27]:
small[0]

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\Wenhao\miniconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Wenhao\AppData\Local\Temp\ipykernel_20660\1814949520.py", line 1, in <module>
    small[0]
TypeError: 'PipelinedRDD' object is not subscriptable

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\Wenhao\miniconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2102, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "c:\Users\Wenhao\miniconda3\lib\site-packages\IPython\core\ultratb.py", line 1310, in structured_traceback
    return FormattedTB.structured_traceback(
  File "c:\Users\Wenhao\miniconda3\lib\site-packages\IPython\core\ultratb.py", line 1199, in structured_traceback
    return VerboseTB.structured_traceback(
  File "c:\Users\Wenhao\miniconda3\lib\site-packag

In [15]:
a = np.zeros((1000, 1000))

In [18]:
small[0, 0]

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\Wenhao\miniconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Wenhao\AppData\Local\Temp\ipykernel_20660\2200473566.py", line 1, in <module>
    small[0, 0]
  File "c:\Users\Wenhao\miniconda3\lib\site-packages\pyspark\sql\dataframe.py", line 3083, in __getitem__
    return self.select(*item)
  File "c:\Users\Wenhao\miniconda3\lib\site-packages\pyspark\sql\dataframe.py", line 3227, in select
    jdf = self._jdf.select(self._jcols(*cols))
  File "c:\Users\Wenhao\miniconda3\lib\site-packages\pyspark\sql\dataframe.py", line 2764, in _jcols
    return self._jseq(cols, _to_java_column)
  File "c:\Users\Wenhao\miniconda3\lib\site-packages\pyspark\sql\dataframe.py", line 2751, in _jseq
    return _to_seq(self.sparkSession._sc, cols, converter)
  File "c:\Users\Wenhao\miniconda3\lib\site-packages\pyspark\sql\column.py", line 88, in _to_seq
   